<a href="https://colab.research.google.com/github/Kopfgeldjaeger/Medium_blogs_code/blob/master/5_covid19checker/tell_if_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 ! pip install -q kaggle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
#download dataset from kaggle
!kaggle datasets download -d iamhungundji/covid19-symptoms-checker

  0% 0.00/912k [00:00<?, ?B/s]
100% 912k/912k [00:00<00:00, 59.8MB/s]


In [ ]:
#unzip the dataset 
!unzip \*.zip  && rm *.zip

Archive:  covid19-symptoms-checker.zip
  inflating: Cleaned-Data.csv        
  inflating: Raw-Data.csv            


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
df= pd.read_csv("Cleaned-Data.csv",header=0)
df

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,Other
316796,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,Other
316797,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,Other
316798,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,Other


In [6]:
print(np.unique(df['Country'].values))

['China' 'France' 'Germany' 'Iran' 'Italy' 'Other' 'Other-EUR'
 'Republic of Korean' 'Spain' 'UAE']


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316800 entries, 0 to 316799
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Fever                    316800 non-null  int64 
 1   Tiredness                316800 non-null  int64 
 2   Dry-Cough                316800 non-null  int64 
 3   Difficulty-in-Breathing  316800 non-null  int64 
 4   Sore-Throat              316800 non-null  int64 
 5   None_Sympton             316800 non-null  int64 
 6   Pains                    316800 non-null  int64 
 7   Nasal-Congestion         316800 non-null  int64 
 8   Runny-Nose               316800 non-null  int64 
 9   Diarrhea                 316800 non-null  int64 
 10  None_Experiencing        316800 non-null  int64 
 11  Age_0-9                  316800 non-null  int64 
 12  Age_10-19                316800 non-null  int64 
 13  Age_20-24                316800 non-null  int64 
 14  Age_25-59           

In [8]:
display("Description",df.describe())

'Description'

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes
count,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000,316800.000000
mean,0.312500,0.500000,0.562500,0.500000,0.312500,0.062500,0.363636,0.545455,0.545455,0.363636,0.090909,0.200000,0.200000,0.200000,0.200000,0.200000,0.333333,0.333333,0.333333,0.250000,0.250000,0.250000,0.250000,0.333333,0.333333,0.333333
std,0.463513,0.500001,0.496079,0.500001,0.463513,0.242062,0.481046,0.497930,0.497930,0.481046,0.287480,0.400001,0.400001,0.400001,0.400001,0.400001,0.471405,0.471405,0.471405,0.433013,0.433013,0.433013,0.433013,0.471405,0.471405,0.471405
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.500000,1.000000,0.500000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.250000,0.250000,0.250000,0.250000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df['Country'].value_counts()

Germany               31680
Italy                 31680
Other-EUR             31680
China                 31680
Iran                  31680
UAE                   31680
France                31680
Other                 31680
Spain                 31680
Republic of Korean    31680
Name: Country, dtype: int64

# feature column preprocessing

In [10]:
country_id = {"Country":     {"China": 0, "UAE": 1, 'Italy':2,'Spain':3,'Iran':4,'Republic of Korean':5,
                              'France':6,'Germany':7,'Other-EUR':8,'Other':9}}
df = df.replace(country_id)
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
2,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
3,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0
4,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0


In [11]:

df['Age'] = df['Age_0-9']*0+df['Age_10-19']*1+df['Age_20-24']*2+df['Age_25-59']*3+df['Age_60+']*4
df

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Age
0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0
2,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0
3,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
4,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,9,4
316796,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,9,4
316797,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,9,4
316798,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,9,4


In [12]:
df['Gender'] = df['Gender_Female']*0+df['Gender_Male']*1+df['Gender_Transgender']*2
df['Contact']= df['Contact_Dont-Know']*1+df['Contact_No']*0+ df['Contact_Yes']*2
df['Severity']= df['Severity_None']*0+df['Severity_Mild']*1+df['Severity_Moderate']*2+ df['Severity_Severe']*3
df= df.drop(columns=['Gender_Female', 'Gender_Male','Gender_Transgender','Severity_None','Severity_Mild','Severity_Moderate','Severity_Severe',
                     'Age_0-9','Age_10-19','Age_20-24','Age_25-59','Age_60+','Contact_Dont-Know','Contact_No', 'Contact_Yes'])
df

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Country,Age,Gender,Contact,Severity
0,1,1,1,1,1,0,1,1,1,1,0,0,0,1,2,1
1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,1
2,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1
3,1,1,1,1,1,0,1,1,1,1,0,0,0,1,2,2
4,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,1,9,4,2,0,3
316796,0,0,0,0,0,1,0,0,0,0,1,9,4,2,1,3
316797,0,0,0,0,0,1,0,0,0,0,1,9,4,2,2,0
316798,0,0,0,0,0,1,0,0,0,0,1,9,4,2,0,0


# train and test data split

In [13]:
np.random.seed(1)
train=df.sample(frac=0.8,random_state=1) 
test=df.drop(train.index)

In [14]:
print(train.columns[:-1])

Index(['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing',
       'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion',
       'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Country', 'Age',
       'Gender', 'Contact'],
      dtype='object')


In [15]:
prior = (train.groupby('Severity').size()).div(len(train)) 
print(prior)
print(type(prior))
likelihood = {}

for x in train.columns[:-1]:
  likelihood[x] = (train.groupby(['Severity', x]).size()).div(len(train)).div(prior)
print (likelihood)

Severity
0    0.250087
1    0.249842
2    0.250114
3    0.249957
dtype: float64
<class 'pandas.core.series.Series'>
{'Fever': Severity  Fever
0         0        0.687167
          1        0.312833
1         0        0.687887
          1        0.312113
2         0        0.687880
          1        0.312120
3         0        0.688251
          1        0.311749
dtype: float64, 'Tiredness': Severity  Tiredness
0         0            0.498990
          1            0.501010
1         0            0.499447
          1            0.500553
2         0            0.499740
          1            0.500260
3         0            0.500560
          1            0.499440
dtype: float64, 'Dry-Cough': Severity  Dry-Cough
0         0            0.437743
          1            0.562257
1         0            0.437761
          1            0.562239
2         0            0.437773
          1            0.562227
3         0            0.438128
          1            0.561872
dtype: float64, 'Difficu

In [16]:
display(test)
test_case = test.to_numpy()
print(test_case)

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Country,Age,Gender,Contact,Severity
0,1,1,1,1,1,0,1,1,1,1,0,0,0,1,2,1
5,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,2
7,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0,3
26,1,1,1,1,1,0,1,1,0,0,0,0,0,1,1,1
27,1,1,1,1,1,0,1,1,0,0,0,0,0,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316786,0,0,0,0,0,1,0,0,0,1,0,9,4,2,0,0
316793,0,0,0,0,0,1,0,0,0,0,1,9,4,2,1,2
316794,0,0,0,0,0,1,0,0,0,0,1,9,4,2,2,3
316795,0,0,0,0,0,1,0,0,0,0,1,9,4,2,0,3


[[1 1 1 ... 1 2 1]
 [1 1 1 ... 1 1 2]
 [1 1 1 ... 1 0 3]
 ...
 [0 0 0 ... 2 2 3]
 [0 0 0 ... 2 0 3]
 [0 0 0 ... 2 1 3]]


# prediction class define so that we can test the trained model

In [66]:
# define how the classifier works: predict returns list of Probability of the each class (perior p);
# result returns 1 for true prediction and 0 for falsh prediction, variable output is the prediction
# wide_correct states that if only the person infected, there is only infected or none-infected,the prediction value of 1,2,3 represent infected and 0 means none infected;
class example:
  def __init__(self,report):
    self.Fever,self.Tiredness,self.Dry_Cough,self.Difficulty_in_Breathing	,self.Sore_Throat,self.None_Sympton,self.Pains,	self.Nasal_Congestion,	self.Runny_Nose,	self.Diarrhea,	self.None_Experiencing,self.Country, self.Age,	self.Gender,	self.Contact, self.Severity=report[:]
    self.output=0
    self.correct=0
    self.wide_correct=0
  def predict(self,report):
    prediction=[]
    for s in range(4):
      P_list=[]
      for i, m in enumerate(train.columns[:-1]):
        p= likelihood[m].at[s,report[i]]
        P_list.append(p)
      p= prior[s]*np.prod(P_list)
      prediction.append(p)
    return prediction
  def result(self,report):
    pro = self.predict(report)
    self.output =np.argmax(pro)
    if self.Severity == self.output:
      self.correct=1
    elif self.Severity != self.output:
      self.correct=0
    positive_pre= (self.output>0)
    positive_truth = (self.Severity>0)
    self.wide_correct = int(positive_pre==positive_truth) 
    return self.wide_correct, self.correct, self.output


In [67]:
one=example(test_case[100])
print(one.predict(test_case[100]))
print(one.result(test_case[100]))
print(one.Severity)
print(test_case[100])

[1.1117603154746233e-06, 1.1160399651761202e-06, 1.1330431870167347e-06, 1.1283259234136788e-06]
(1, 0, 2)
3
[1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 3]


In [68]:
predictions,true_prediction,wide_prediction=[],[],[]
for i in test_case:
  case =example(i)
  a,b,c =case.result(i)
  wide_prediction.append(a)
  true_prediction.append(b)
  predictions.append(c)


## prediction acc

In [82]:
acc = sum(true_prediction)/len(test_case)*100
wide_acc =sum(wide_prediction)/len(test_case)*100
print("The accuracy of the naive bayes classifier: % 5.2f"%(acc))

The accuracy of the naive bayes classifier:  24.17


In [60]:
print(predictions)
y= [x[-1] for x in test_case[0:1000]]
print(y)

[1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 3, 2, 3, 3, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 3, 2, 3, 3, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 3, 2, 2, 3, 1, 3, 2, 3, 2, 3, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 3, 2, 3, 1, 1, 1, 2, 1, 3, 2, 3, 1, 3, 3, 2, 2, 2, 2, 3, 1, 1, 1, 2, 2, 2, 3, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 3, 2, 1, 1, 2, 1, 1, 3, 1, 1, 2, 3, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 2, 3, 2, 3, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 3, 3, 2, 3, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1, 2, 2, 3, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 3, 3, 1, 3, 2, 3, 3, 2, 2, 3, 2, 1, 1, 1, 3, 2, 2, 3, 3, 3, 2, 1, 1, 1, 1, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 1, 1, 1, 2, 3, 2, 3, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 3, 3, 1, 1, 2, 2, 3, 2, 3, 2, 2, 3, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 

# use Sklearn package to realize naive bayes

In [75]:
x_train = train[train.columns[:-1]] # Features
y_train = train.Severity
x_test =test[train.columns[:-1]]
y_test=test.Severity

In [79]:
from sklearn.naive_bayes import GaussianNB,CategoricalNB
gnb = CategoricalNB()
y_pred = gnb.fit(x_train, y_train).predict(x_test)
print("Number of mislabeled points out of a total %d points : %d"\
      % (x_test.shape[0], (y_test != y_pred).sum()))
print("The accuracy of the naive bayes classifier:%5.2f"%(100*(y_test == y_pred).sum()/x_test.shape[0]))

Number of mislabeled points out of a total 63360 points : 48047
The accuracy of the naive bayes classifier:24.17
